In [2]:

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append('..')
os.environ['CUDA_VISIBLE_DEVICES'] =''
import glob

In [3]:
from nn_ansatz import approximate_pair_distribution_function, load_pk
from nn_ansatz.python_helpers import find_all_files_in_dir
from nn_ansatz.plot import data_and_plot_scatter, get_data, plot_scatter
import numpy as np
from bokeh.io import output_notebook
output_notebook()
from bokeh.palettes import Dark2_5 as palette
from bokeh.plotting import figure, show
import itertools


2021-11-19 08:37:18.573450: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/modules/software/binutils/2.36.1-GCCcore-10.3.0/lib:/home/modules/software/zlib/1.2.11-GCCcore-10.3.0/lib:/home/modules/software/GCCcore/10.3.0/lib64:/home/modules/software/GCCcore/10.3.0/lib
2021-11-19 08:37:20.733243: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/modules/software/binutils/2.36.1-GCCcore-10.3.0/lib:/home/modules/software/zlib/1.2.11-GCCcore-10.3.0/lib:/home/modules/software/GCCcore/10.3.0/lib64:/home/modules/software/GCCcore/10.3.0/lib
2021-11-19 08:37:20.745694: W external/org_tensorflow/tensor

Loading BokehJS ...

In [5]:
filters = ['seed', 'save_every', 'print_every', 'basis', 'load_pretrain', \
        'pretrain', 'n_walkers_per_device', 'n_devices', 'pre_lr', 'n_pre_it', \
            'lr', 'damping', 'norm_constraint', 'correlation_length', 'scalar_inputs', 'einsum']
filters.extend(['real_cut', 'reciprocal_cut', 'kappa'])
filters.extend(['e_mean_mean', 'acceptance'])
filters.extend(['pbc', 'scale_cell', 'spin_polarized', 'r_atoms', 'z_atoms', 'simulation_cell', 'e_mean', 'e_std'])


In [5]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1811/find_act', dicts=[])

data = data[data.equilibrated_energy_mean.notna()]
data = data.sort_values(by=['nonlinearity', 'input_activation_nonlinearity'])
data['xticklabel'] = data['nonlinearity'] + '/' + data['input_activation_nonlinearity']
data_grouped_nonlin = data.groupby('nonlinearity')
data_grouped_inactnonlin = data.groupby('input_activation_nonlinearity')

graph = plot_scatter(list(range(len(data['xticklabel']))), 
             data['equilibrated_energy_mean'], 
             yerrs=data['equilibrated_energy_sem'], \
             xticklabels=data['xticklabel'], 
             title='activations and convergence') 
show(graph)

graph = plot_scatter(list(range(len(data_grouped_nonlin.mean().index))), 
             data_grouped_nonlin['equilibrated_energy_mean'].mean(), 
             yerrs=data_grouped_nonlin['equilibrated_energy_mean'].std(), \
             xticklabels=data_grouped_nonlin.mean().index, 
             title='activations and convergence') 
show(graph)

graph = plot_scatter(list(range(len(data_grouped_inactnonlin.mean().index))), 
             data_grouped_inactnonlin['equilibrated_energy_mean'].mean(), 
             yerrs=data_grouped_inactnonlin['equilibrated_energy_mean'].std(), \
             xticklabels=data_grouped_inactnonlin.mean().index, 
             title='activations and convergence') 
show(graph)



max it: 10000


In [6]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1811/find_n_walkers', dicts=[])

graph = plot_scatter(data['n_walkers'], 
             data['equilibrated_energy_mean'], 
             yerrs=data['equilibrated_energy_sem'],
             title='n_walkers sweep') 

show(graph)

data[['n_walkers', 'equilibrated_energy_mean', 'equilibrated_energy_sem', 'energy (Ry)']].sort_values(by='n_walkers').style.hide_index()

max it: 10000


n_walkers,equilibrated_energy_mean,equilibrated_energy_sem,energy (Ry)
512,1.070863,0.000137,2.141726
1024,1.069311,0.000097,2.138623
2048,1.068603,0.000072,2.137207
4096,1.064937,0.000047,2.129875


In [7]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1811/find_n_params', dicts=[])

graph = plot_scatter(data['n_sh'], 
             data['equilibrated_energy_mean'], 
             yerrs=data['equilibrated_energy_sem'],
             title='n_params sweep') 

show(graph)

data[['n_sh', 'n_ph', 'equilibrated_energy_mean', 'equilibrated_energy_sem', 'energy (Ry)']].sort_values(by='n_sh').style.hide_index()

max it: 10000


n_sh,n_ph,equilibrated_energy_mean,equilibrated_energy_sem,energy (Ry)
32,8,0.950997,0.003308,1.901993
64,16,0.955293,0.002219,1.910587
128,32,0.932497,0.005849,1.864995
256,64,nan,nan,nan


In [12]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1811/find_n_det', dicts=[])

data = data.groupby(by='n_det')
print(data['equilibrated_energy_mean'].mean())
graph = plot_scatter(data['n_det'].mean(), 
             data['equilibrated_energy_mean'].mean(), 
             yerrs=data['equilibrated_energy_sem'].mean(),
             title='n_det sweep') 

show(graph)

# data[['n_det', 'equilibrated_energy_mean', 'equilibrated_energy_sem', 'energy (Ry)']].sort_values(by='n_det').style.hide_index()

max it: 10000
n_det
1     0.945421
2     0.940300
4     0.955121
8     0.958416
16    0.921349
Name: equilibrated_energy_mean, dtype: float32


In [33]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1811/find_jastrow/', dicts=[])

colors = itertools.cycle(palette) 

data_jas = data[data['jastrow'] == True]
graph = plot_scatter(data_jas['density_parameter'],
                data_jas['equilibrated_energy_mean'], 
                yerrs=data['equilibrated_energy_sem'],
                title='el7 para dp sweep',
                line_label='jastrow',
                colors=colors) 

data_jas = data[data['jastrow'] == False]
graph = plot_scatter(data_jas['density_parameter'],
                    data_jas['equilibrated_energy_mean'], 
                    yerrs=data['equilibrated_energy_sem'],
                    title='el7 para dp sweep',
                    line_label='no_jastrow',
                    graph=graph,
                    colors=colors,
                    legend_location='top_right') 
show(graph)
data[['density_parameter', 'equilibrated_energy_mean', 'equilibrated_energy_sem', 'energy (Ry)']].sort_values(by='density_parameter').style.hide_index()

max it: 10000


density_parameter,equilibrated_energy_mean,equilibrated_energy_sem,energy (Ry)
0.500000,5.810587,0.000083,11.621175
0.500000,5.621586,0.000311,11.243172
1.000000,1.121217,0.000040,2.242433
1.000000,1.066168,0.000098,2.132335
2.000000,0.111613,0.000019,0.223227
2.000000,0.098789,0.000030,0.197579
5.000000,-0.065848,0.000007,-0.131697
5.000000,-0.068274,0.000008,-0.136549
10.000000,-0.052719,0.000003,-0.105438
10.000000,-0.053055,0.000004,-0.106111


density_parameter,equilibrated_energy_mean_i10000,equilibrated_energy_sem_i10000,energy (Ry)
0.100000,171.440002,0.000127,342.880005
0.200000,41.231842,0.000064,82.463684
0.500000,5.810611,0.000025,11.621222
1.000000,1.121276,0.000012,2.242551
2.000000,0.111633,0.000006,0.223267
5.000000,-0.065857,0.000002,-0.131715
10.000000,-0.052750,0.000001,-0.105500


In [18]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1611/el19_dp_sweep', dicts=[])

plot_scatter(data['density_parameter'], 
             data['equilibrated_energy_mean_i10000'], 
             yerrs=data['equilibrated_energy_sem_i10000'],
             title='density parameter sweep') 

data['energy (Ry)'] = data['equilibrated_energy_mean_i10000'] * 2.
data[['density_parameter', 'equilibrated_energy_mean_i10000', 'equilibrated_energy_sem_i10000', 'energy (Ry)']].sort_values(by='density_parameter').style.hide_index()

density_parameter,equilibrated_energy_mean_i10000,equilibrated_energy_sem_i10000,energy (Ry)
0.100000,162.203156,0.000089,324.406311
0.200000,38.980652,0.000045,77.961304
0.500000,5.488117,0.000046,10.976234
2.000000,0.097371,0.000004,0.194742
5.000000,-0.066564,0.000002,-0.133129
10.000000,-0.048785,0.000002,-0.097570


In [14]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1511/7el_opt_sweep', dicts=[])

xs = [i for i in range(5000, 100000, 10000)]
ys = [data['equilibrated_energy_mean_i%i' % i].values for i in xs if 'equilibrated_energy_mean_i%i' % i in data.columns]
errs = [data['equilibrated_energy_sem_i%i' % i].values for i in xs if 'equilibrated_energy_mean_i%i' % i in data.columns]
min_y = min(ys)
ys = [y - min_y for y in ys]
plot_scatter(xs, 
             ys, 
             yerrs=errs,
             title='training',
             xaxis='log', yaxis='log')

In [4]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1311/inact_act_sweep2', dicts=[])



columns = [x for x in data.columns if x not in filters]

data = data[columns]
data = data[data.equilibrated_energy_mean.notna()]
data = data.sort_values(by=['nonlinearity', 'input_activation_nonlinearity'])
data['xticklabel'] = data['nonlinearity'] + '/' + data['input_activation_nonlinearity']
data_grouped_nonlin = data.groupby('nonlinearity')
data_grouped_inactnonlin = data.groupby('input_activation_nonlinearity')


In [67]:

plot_scatter(list(range(len(data['xticklabel']))), 
             data['equilibrated_energy_mean'], 
             yerrs=data['equilibrated_energy_sem'], \
             xticklabels=data['xticklabel'], 
             title='activations and convergence') 

plot_scatter(list(range(len(data_grouped_nonlin.mean().index))), 
             data_grouped_nonlin['equilibrated_energy_mean'].mean(), 
             yerrs=data_grouped_nonlin['equilibrated_energy_mean'].std(), \
             xticklabels=data_grouped_nonlin.mean().index, 
             title='activations and convergence') 

plot_scatter(list(range(len(data_grouped_inactnonlin.mean().index))), 
             data_grouped_inactnonlin['equilibrated_energy_mean'].mean(), 
             yerrs=data_grouped_inactnonlin['equilibrated_energy_mean'].std(), \
             xticklabels=data_grouped_inactnonlin.mean().index, 
             title='activations and convergence') 



In [69]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1311/det_sweep', dicts=[])
columns = [x for x in data.columns if x not in filters]

data = data[columns]
data = data[data.equilibrated_energy_mean.notna()]
data = data.sort_values(by=['n_det',])

plot_scatter(data['n_det'], 
             data['equilibrated_energy_mean'], 
             yerrs=data['equilibrated_energy_sem'], 
             title='activations and convergence') 


In [12]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1511/el7_dp_sweep', dicts=[])

plot_scatter(data['density_parameter'], 
             data['equilibrated_energy_mean'], 
             yerrs=data['equilibrated_energy_sem'],
             title='activations and convergence') 

data['energy (Ry)'] = data['equilibrated_energy_mean'] * 2.
data[['density_parameter', 'equilibrated_energy_mean', 'equilibrated_energy_sem', 'energy (Ry)']].sort_values(by='density_parameter').style.hide_index()

density_parameter,equilibrated_energy_mean,equilibrated_energy_sem,energy (Ry)
0.500000,25.865519,0.000035,51.731037
1.000000,1.121301,0.000009,2.242602
1.500000,0.054360,0.000004,0.108721
2.000000,-0.058830,0.000002,-0.117661
2.500000,-0.065128,0.000001,-0.130257
3.000000,-0.056134,0.000001,-0.112269
3.500000,-0.046357,0.000001,-0.092713
4.000000,-0.038218,0.000000,-0.076437
4.500000,-0.020712,0.000149,-0.041423
5.000000,0.026916,0.000078,0.053831


In [72]:
data = get_data('/home/energy/amawi/projects/nn_ansatz/src/experiments/HEG/1311/param_sweep', dicts=[])
columns = [x for x in data.columns if x not in filters]

data = data[columns]
data = data[data.equilibrated_energy_mean.notna()]
data = data.sort_values(by=['n_sh',])

plot_scatter(data['n_sh'] / 32, 
             data['equilibrated_energy_mean'], 
             yerrs=data['equilibrated_energy_sem'], 
             title='param sweep') 


In [9]:
# plot an optimisation plot
target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/051121/adam_vs_kfac'
data_paths = find_all_files_in_dir(target_dir, 'data.pk')
names = ['adam', 'kfac']

colors = itertools.cycle(palette) 

min = 100.
for name, data_path in zip(names, data_paths):
    data = load_pk(data_path)
    e_means = data['e_mean_mean']
    data_min = np.min(e_means)
    min = data_min if data_min < min else min

graph = figure(title = "adam_vs_kfac_opt", 
               x_axis_label='iteration', y_axis_label='energy (shifted by min for log)',
               width=400, height=400, 
               x_axis_type='log', y_axis_type='log')

for name, data_path in zip(names, data_paths):
    data = load_pk(data_path)
    e_means = np.array(data['e_mean_mean']) - min
    print(np.min(e_means))
    x = range(1, len(e_means)+1)
    graph.line(x, e_means, legend_label=name, color=next(colors))

graph.legend.location = 'top_right'
show(graph)

0.09865474700927734
0.0


In [ ]:
# plot a sweep
target_dirs = ['/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/1112/inact_act%s' % s for s in ['', '1', '2']]
data_filename = 'config1.pk'
hypam = 'density_parameter'
xname = 'equilibrated_energy_mean'
errname = 'equilibrated_energy_sem'

df = plot_scatter(target_dir, data_filename, hypam, xname, errname, 
title='converged energy vs density parameter', xlabel='density parameter', ylabel='energy')
df

target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/bowl_density_sweep_HEG/kfac_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det1'


In [10]:
# plot a sweep
target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/051121/density_sweep_HEG'
data_filename = 'config1.pk'
hypam = 'density_parameter'
xname = 'equilibrated_energy_mean'
errname = 'equilibrated_energy_sem'

df = plot_scatter(target_dir, data_filename, hypam, xname, errname, 
title='converged energy vs density parameter', xlabel='density parameter', ylabel='energy')
df

target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/bowl_density_sweep_HEG/kfac_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det1'


,r_s,energy,energy_sem
0,0.5,181.091827,0.004311
1,1.0,7.906222,0.012094
2,2.0,-0.301703,0.022778
3,3.0,-0.261176,0.021507
4,4.0,-0.042468,0.018287
5,5.0,-0.028535,0.014086


In [19]:
data_filename = 'config1.pk'
hypam = 'density_parameter'
xname = 'equilibrated_energy_mean'
errname = 'equilibrated_energy_sem'
target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/bowl_density_sweep_HEG'
df = plot_scatter(target_dir, data_filename, hypam, xname, errname, 
title='converged energy vs density parameter', xlabel='density parameter', ylabel='energy')
df.sort_values(by='r_s')

,r_s,energy,energy_sem
0,0.5,181.073471,0.002536
2,1.0,7.844774,0.004103
4,2.0,-0.306160,0.032655
1,3.0,-0.180535,0.032855
3,4.0,0.080237,0.022115
5,5.0,0.096446,0.016455


In [11]:
# plot a sweep
target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/051121/parameter_ablation_HEG'
data_filename = 'config1.pk'
hypam = 'n_sh'
xname = 'equilibrated_energy_mean'
errname = 'equilibrated_energy_sem'

plot_scatter(target_dir, data_filename, hypam, xname, errname, title='converged energy vs n_hidden (n_pairwsise increasing also)', xlabel='n_hidden', ylabel='energy')

,r_s,energy,energy_sem
0,8,7.900940,0.009994
1,16,7.904780,0.011611
2,32,7.905794,0.011925
3,64,7.902023,0.011261


In [15]:
# plot a sweep
# plot an optimisation plot
target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/LiSolidBCC/051121/input_activation_nonlinearity_compare_Li'
data_filename = 'config1.pk'
hypam = 'input_activation_nonlinearity'
yname = 'equilibrated_energy_mean'
errname = 'equilibrated_energy_sem'

plot_scatter(target_dir, data_filename, hypam, yname, errname, 
             title='converged energy vs inact_nonlinearity (Li)', 
             xlabel='inact_nonlinearity', 
             ylabel='energy',
             hlines=[-14.55])

,r_s,energy,energy_sem
0,0,-14.569840,0.010703
1,1,-14.329383,0.008107


In [6]:
target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/101121/inact_sweep'
data_filename = 'config1.pk'
hypam = 'input_activation_nonlinearity'
yname = 'equilibrated_energy_mean_per_particle'
errname = 'equilibrated_energy_sem_per_particle'

df = plot_scatter(target_dir, data_filename, hypam, yname, errname, 
             title='converged energy vs inact_nonlinearity (HEG)', 
             xlabel=hypam, 
             ylabel=yname)

df

,input_activation_nonlinearity,equilibrated_energy_mean_per_particle,yerr
0,bowl,1.114686,0.000057
1,sin,1.182655,0.000088
2,sin+cos,1.137414,0.000052
3,sin+cos+bowl,1.115609,0.000054
4,2sin+2cos,1.137970,0.000057


In [22]:
# plot a sweep
target_dir = '/home/amawi/projects/nn_ansatz/src/exp/experiments/HSolid/H_sim_cell_sweep'
data_filename = 'config1.pk'
hypam = 'simulation_cell'
yname = 'equilibrated_energy_mean_per_particle'
# errname = 'equilibrated_energy_sem'  # doesn't work because bug

plot_scatter(target_dir, data_filename, hypam, yname, errname=None, 
             title='Extrapolation over increasing lattice size (H)', 
             xlabel='lattice', 
             ylabel='energy')

'equilibrated_energy_mean_per_particle'


ValueError: All arrays must be of the same length

In [9]:
pdf_saverio = np.loadtxt('/home/amawi/projects/nn_ansatz/src/exp/gofr.txt')[:, 0]

In [26]:
cfg = load_pk('/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/051121/adam_vs_kfac/kfac_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det1/run0/config1.pk')

pdf, pdf_std = approximate_pair_distribution_function(cfg, load_it=50000, n_bins=len(pdf_saverio)-1, n_it=25, walkers=None)
save_me = np.stack([pdf, pdf_std, pdf_saverio], axis=-1)
np.savetxt('/home/amawi/projects/nn_ansatz/src/exp/gofr_max.txt', save_me)

RuntimeError: jaxlib/cusolver.cc:51: operation cusolverDnCreate(&handle) failed: cuSolver internal error

In [69]:
cfg = load_pk('/home/amawi/projects/nn_ansatz/src/exp/experiments/HEG/bowl_density_sweep_HEG/kfac_1lr-4_1d-3_1nc-4_m512_s64_p32_l2_det1/run1/config1.pk')
n_bins = 100
n_bins += 1
load_it = 25000
walkers = None
n_it = 100
from nn_ansatz import initialise_system_wf_and_sampler, equilibrate, split_variables_for_pmap, drop_diagonal_i, apply_minimum_image_convention, key_gen
from jax import vmap
import jax.numpy as jnp


if load_it is not None:
    cfg['load_it'] = load_it

mol, vwf, walkers, params, sampler, keys = initialise_system_wf_and_sampler(cfg, walkers)
n_walkers, n_el = walkers.shape[:2]
bins = np.linspace(0., mol.scale_cell/2., n_bins, endpoint=False)
print(bins)

walkers = equilibrate(params, walkers, keys, mol=mol, vwf=vwf, sampler=sampler, compute_energy=True, n_it=1000)

step_size = split_variables_for_pmap(cfg['n_devices'], cfg['step_size'])
drop_diagonal = vmap(drop_diagonal_i, in_axes=(0,))

binned_ee_distances_list = []
for i in range(n_it):
    keys, subkeys = key_gen(keys)

    walkers, acceptance, step_size = sampler(params, walkers, subkeys, step_size)
    ee_vectors = walkers[:, None, ...] - walkers[:, :, None, ...]
    # ee_vectors = apply_minimum_image_convention(ee_vectors, mol.basis, mol.inv_basis)
    # ee_vectors = ee_vectors[:, 0, 1:, :]
    ee_vectors = drop_diagonal(ee_vectors)  # (n_walkers, n_el - 1, 3)

    ee_distances = np.array(jnp.linalg.norm(ee_vectors, axis=-1)).reshape(-1)
    # binned_dists = np.digitize(ee_distances, bins=bins).reshape(-1) # divide by r**2
    # bin_freqs = (np.bincount(binned_dists, minlength=(n_bins+1)) / (float(n_walkers)))[:-1]
    counts, _ = np.histogram(ee_distances, bins=n_bins, range=(bins.min(), bins.max()))
    g_r = counts * 2 / (n_el * 4 * jnp.pi * mol.density_parameter * bins**2)

    binned_ee_distances_list.append(g_r)

    if i % 10 == 0:
        print('step %i ' % i)

binned_ee_distances = jnp.array(binned_ee_distances_list)
pdf = jnp.mean(binned_ee_distances, axis=0)
pdf_std = jnp.std(binned_ee_distances, axis=0) / jnp.sqrt(len(binned_ee_distances))


Cell: 
 basis: 
 [[9.250889 9.250889 9.250889]] 
 inv_basis: 
 [[0.10809772 0.10809772 0.10809772]] 
 reciprocal_basis: 
 [[0.679198 0.       0.      ]
 [0.       0.679198 0.      ]
 [0.       0.       0.679198]] 
 real_cut         = 6.00 
 reciprocal_cut   = 6 
 kappa            = 0.50 
 volume           = 791.68 
 n_periodic_input = 1 

System: 
 n_atoms = 0 
 n_up    = 7 
 n_down  = 0 
 n_el    = 7 

Ansatz: 
 n_layers = 2 
 n_det    = 1 
 n_sh     = 64 
 n_ph     = 32 

creating wf
sampling no infs, this could take a while
end sampling no infs
[0.         0.04579648 0.09159296 0.13738944 0.18318592 0.2289824
 0.27477888 0.32057536 0.36637184 0.41216832 0.4579648  0.50376128
 0.54955776 0.59535424 0.64115072 0.6869472  0.73274369 0.77854017
 0.82433665 0.87013313 0.91592961 0.96172609 1.00752257 1.05331905
 1.09911553 1.14491201 1.19070849 1.23650497 1.28230145 1.32809793
 1.37389441 1.41969089 1.46548737 1.51128385 1.55708033 1.60287681
 1.64867329 1.69446977 1.74026625 1.78606273 

ipykernel_launcher:40: RuntimeWarning: invalid value encountered in true_divide


step 10 
step 20 
step 30 
step 40 
step 50 
step 60 
step 70 


ipykernel_launcher:40: RuntimeWarning: divide by zero encountered in true_divide


step 80 
step 90 


In [70]:
print(np.sum(bin_freqs), walkers.shape, mol.basis)

5722 (512, 7, 3) [[9.250889 0.       0.      ]
 [0.       9.250889 0.      ]
 [0.       0.       9.250889]]


In [71]:
print(ee_distances.shape)
print(ee_distances.max(), ee_distances.min())
print(((mol.scale_cell/2.)**2 + (mol.scale_cell/2.)**2 + (mol.scale_cell/2.)**2)**0.5)

np.histogram(ee_distances, bins=n_bins, range=(bins.min(), bins.max()))

(21504,)
13.784223 0.22065201
8.011504903802619


(array([  0,   0,   0,   0,   2,   0,   2,   0,   4,   6,   0,   4,   4,
          8,   4,   8,   2,  16,  14,  18,  14,  18,  14,  18,  14,   8,
         22,  30,  12,  12,  32,  30,  36,  20,  18,  18,  24,  28,  44,
         40,  38,  44,  32,  40,  48,  60,  46,  36,  44,  44,  44,  66,
         58,  62,  74,  64,  44,  76,  58,  78,  72,  64,  76,  88,  80,
         92,  92,  78,  80,  84,  70,  68,  82,  88,  82, 100,  66,  90,
         96, 122,  88,  88,  98,  82, 122, 114, 120, 134, 124, 124, 146,
        104, 106, 150, 124, 134, 124, 134, 128, 102, 106]),
 array([0.        , 0.04534305, 0.0906861 , 0.13602915, 0.1813722 ,
        0.22671525, 0.2720583 , 0.31740135, 0.3627444 , 0.40808746,
        0.4534305 , 0.49877355, 0.5441166 , 0.58945966, 0.6348027 ,
        0.68014574, 0.7254888 , 0.7708318 , 0.8161749 , 0.86151797,
        0.906861  , 0.95220405, 0.9975471 , 1.0428902 , 1.0882332 ,
        1.1335763 , 1.1789193 , 1.2242624 , 1.2696054 , 1.3149484 ,
        1.3602915 , 1

In [72]:
from bokeh.plotting import figure, output_file, show

 
# instantiating the figure object
graph = figure(title = "pdf", width=250, height=250)
 
 
# plotting the line graph
x = range(len(pdf))
pdf = np.where(np.isnan(pdf), 0.0, pdf)
graph.line(x, pdf, color='red', legend_label='nn_ansatz')
graph.line(x, pdf_saverio, legend_label='saverio')
graph.legend.location = 'top_left'

# displaying the model
show(graph)

graph = figure(title = "pdf ratio saverio/nn_ansatz", width=250, height=250)
ratio = pdf_saverio / pdf
graph.line(x, np.where(np.isnan(ratio), 0.0, ratio))
graph.legend.location = 'top_left'
show(graph)

ipykernel_launcher:19: RuntimeWarning: invalid value encountered in true_divide
/home/amawi/.conda/envs/xmax/lib/python3.7/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


In [22]:
print(mol.scale_cell/2.)

4.6254445128244495
